In [1]:
from model import *
from data import *

ModuleNotFoundError: ignored

### Load Colab Notebook settings

**Link Google Drive**

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Try GPU power**

In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=a8498d7a091ffd53c7cf65b85a580759b1ccbdb2116246b1f5b4d45ae5b0c2ea
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.5 GB  | Proc size: 586.5 MB
GPU RAM Free: 7496MB | Used: 115MB | Util   2% | Total 7611MB


In [0]:
#!kill -9 -1

## Train the model

In [8]:
# mettre dans un .py qaund on a trouvé les bons params

import numpy as np
import sys
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
from keras.preprocessing.image import img_to_array
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as keras

def data_load(directory_name): 
    filenames = os.listdir(directory_name)
    imgs = []
    for i, fileNb in enumerate(filenames):
        if(fileNb[0]!='.'):
            full_name = directory_name+fileNb
            img=mpimg.imread(full_name)
            imgr = img_to_array(img)
            imgs.append(imgr)
            sys.stdout.write("\rImage {}/{} is being loaded".format(i+1,len(filenames)))
            sys.stdout.flush()    
    return np.asarray(imgs)

'''def dataGenerator(batch_size, data, labels, aug_dict, image_save_prefix ="image", mask_save_prefix="mask", seed=1):
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)

    image_generator = image_datagen.flow(
        data,
        batch_size = batch_size,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow(
        labels,
        batch_size = batch_size,
        save_prefix  = mask_save_prefix,
        seed = seed)
    
    for (img,mask) in zip(image_generator, mask_generator):
        if(np.max(img) > 1):
          img = img / 255
          mask = mask /255
          mask[mask > 0.5] = 1
          mask[mask <= 0.5] = 0
        yield (img,mask)'''

# target_size = (256,256)
def dataGenerator(batch_size, train_path, image_folder, mask_folder, aug_dict, target_size,
                  image_color_mode = "rgb", mask_color_mode = "grayscale",
                  image_save_prefix  = "image",mask_save_prefix  = "mask",
                  flag_multi_class = False,num_class = 2,save_to_dir = None, seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        if(np.max(img) > 1):
            img = img / 255
            mask = mask /255
            mask[mask > 0.5] = 1
            mask[mask <= 0.5] = 0
        yield (img,mask)

def uneti(input_size = (256,256,3), pretrained_weights=None, verbose=True):
    inputs = Input(shape=input_size)

    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)
    
    ##binary_crossentropy
    ##categorical_crossentropy
    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    if(verbose == True):
      model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

Using TensorFlow backend.


**1. Load the data and split it into validation and training**

In [17]:
# unet400
#imgs = data_load(root_path+'data_rs/train/images/')
#gts = data_load(root_path+'data_rs/train/groundtruth/')

# unet256 nous
'''from PIL import Image
def data_load_resize(directory_name, size):
    filenames = os.listdir(directory_name)
    imgs = []
    for i, fileNb in enumerate(filenames):
        if(fileNb!='.DS_Store'):     
            im1 = Image.open(directory_name+fileNb)
            side = size
            im2 = im1.resize((side, side), Image.NEAREST)
            imgr = img_to_array(im2)
            imgs.append(imgr)
            sys.stdout.write("\rImage {}/{} is being loaded".format(i+1,len(filenames)))
            sys.stdout.flush()    
    return np.asarray(imgs)

imgs = data_load_resize(root_path+'data_rs/train/images/', 256)
gts = data_load_resize(root_path+'data_rs/train/groundtruth/', 256)

TEST_SIZE = 0.2
X_train, X_test, gt_train, gt_test = train_test_split(imgs, gts, test_size=TEST_SIZE, random_state=1)'''

#unet 256 git
#nothing car c'est fait dans le generator

Image 100/100 is being loaded

**2. Data augmentation and generators**

In [0]:
data_gen_args = dict(rotation_range=0.15,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')



#unet 256 nous
#train_generator = dataGenerator(2, X_train, gt_train, data_gen_args)
#validation_generator = dataGenerator(2, X_test, gt_test, data_gen_args)

#unet 256 git
train_generator = dataGenerator(2,root_path+'data_rs/train','images','groundtruth',data_gen_args,(256,256))


i = 0
for (img,mask) in myGene: 
    i = i+1
    print(i, img.shape, mask.shape)

**3. Load the model**

In [10]:
model = uneti(input_size = (256,256,3), pretrained_weights=None, verbose=True)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       conv2d_1[0][0]                   
________________________________________________________________________

**4. Train the model**

In [12]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=2000,
                    epochs=5, 
                    verbose=1,
                    #validation_data = validation_generator,
                    #validation_steps = 700
                    )




Epoch 1/5
Found 100 images belonging to 1 classes.
Found 100 images belonging to 1 classes.





2000/2000 [==============================] - 515s 258ms/step - loss: 0.2024 - acc: 0.9148
Epoch 2/5
2000/2000 [==============================] - 475s 238ms/step - loss: 0.0896 - acc: 0.9626
Epoch 3/5
2000/2000 [==============================] - 474s 237ms/step - loss: 0.0558 - acc: 0.9762
Epoch 4/5
2000/2000 [==============================] - 472s 236ms/step - loss: 0.0529 - acc: 0.9779
Epoch 5/5
2000/2000 [==============================] - 471s 236ms/step - loss: 0.0330 - acc: 0.9858


### Save model to disk

In [13]:
# serialize model to JSON
OUTPUT_FILENAME = "Model_UNET_256_acc98"

model_json = model.to_json()
with open(OUTPUT_FILENAME + ".json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights(OUTPUT_FILENAME + ".h5")
print("Saved model to disk")

Saved model to disk


### Predictions (don't run it on Colab)

**Load model** [Optional]

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
from keras.models import model_from_json
json_file = open('simple_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

**Resize images in a folder** [Optional]

In [0]:
from PIL import Image
path1 = "##UNET/data/membrane/tmp/"
path2 = "##UNET/data/membrane/test/"

filenames = os.listdir(path1)
for i, fileNb in enumerate(filenames):
    if(fileNb!='.DS_Store'):        
        im1 = Image.open(path1+fileNb)
        side = 400
        # use one of these filter options to resize the image
        im2 = im1.resize((side, side), Image.NEAREST)
        im2.save(path2+fileNb)

FileNotFoundError: ignored

In [0]:
import skimage.io as io
import skimage.transform as trans


'''def testGeneratorNEW(batch_size, train_path, image_folder, aug_dict, image_save_prefix ="prediction",
                     num_class=2, save_to_dir=None, target_size=(256,256), seed=1):
    image_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    for img in image_generator:
        img = img / 255
        yield img'''

def testGeneratorNEW2(batch_size, data, aug_dict, image_save_prefix ="image", mask_save_prefix="mask", seed=1):
  image_datagen = ImageDataGenerator(**aug_dict)

  image_generator = image_datagen.flow(
      data,
      batch_size = batch_size,
      save_prefix  = image_save_prefix,
      seed = seed)
  
  for img in image_generator:
      if(np.max(img) > 1):
        img = img / 255
      yield img

In [0]:

def model_UNETOP(input_size = (256,256,3, 1), pretrained_weights=None):
    inputs = Input(shape=(256,256,3))

    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(3, 1, activation = 'sigmoid')(conv9)

    

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model


In [39]:
#test_generatorNEW = testGeneratorNEW(2, 'UNET/data/membrane','test',data_gen_args,save_to_dir = None)

data_test = data_load(root_path+'data_rs/test/')

test_generatorNEW = testGeneratorNEW2(2, data_test, data_gen_args)

#testGene = testGenerator("UNET/data/membrane/test")


#test_datagen = ImageDataGenerator(rescale=1./255)
#
#testGene = test_datagen.flow_from_directory(
#        "UNET/data/membrane/test/",
#        target_size=(400, 400),
#        color_mode="rgb",
#        shuffle = False,
#        class_mode='categorical',
#        batch_size=1)

print('\n1')

model = uneti(input_size = (256,256,3), pretrained_weights=None, verbose=False)
print('2')

model.load_weights("Model_UNET_256_acc80.h5")
print('3')

results = model.predict_generator(test_generatorNEW, 30,verbose=1)
print('4')

Image 29/30 is being loaded
1
2
3


ValueError: ignored

In [0]:
def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

In [0]:
saveResult("UNET/data/membrane/test",results)